In [95]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
%precision 4

import pyjags
import numpy as np
from scipy.stats import poisson, expon, gamma, bernoulli, norm

def summary(samples, varname, p=95, burnin=0, thin=1):
    values = samples[varname]
    (nb, iter, chains) = values.shape
    
    for i in range(nb):
        data = values[i][burnin::thin]

        ci = np.percentile(data, [100-p, p])
        
        print('{:<8}[{}] mean = {:>12.4f} (std = {:>10.4f}), {}% credible interval [{:>5.2f} {:>5.2f}]'.format(
          varname, i, np.mean(data), np.std(data), p, *ci))

In [98]:
def gen_poisson(T, lambdas):
    return np.array([poisson.rvs(l, size=T) for l in lambdas])

def gen_lambdas(N, n):
    return N * np.abs(norm.rvs(1, 3, size=n))

def gen_noised_poisson(T, lambdas, p_noise=0.05, i_noise=2):
    poissons = gen_poisson(T, lambdas)
    noise = np.array([bernoulli.rvs(p_noise, size=T) * i_noise for l in lambdas])
    return poissons * (1 + noise)

In [99]:
gen_poisson(3, [10, 100]), gen_lambdas(10, 4), gen_noised_poisson(3, gen_lambdas(10,4), p_noise=0.3, i_noise=10)

(array([[ 17,   5,  10],
        [ 88, 104, 113]]),
 array([ 33.8624,  37.0324,  32.6711,   1.0717]),
 array([[  1,   3,   1],
        [ 48,  38,  39],
        [ 16,  20,  18],
        [341,  33,  30]]))

In [2]:
sample = np.array([
        [10, 5, 12, 14, 30],
        [20, 14, 18, 22, 50],
        [20, 14, 18, 15, 25],
        [3, 2, 1, 2, 0]
    ])

cp, T = sample.shape

print(cp, T)

code = """
# bugs model for poisson distribs

model{
        for (i in 1:cp) {
            for (t in 1:T) {
                n[i, t] ~ dpois(a[i] * N[t])
            }
        }
        
        for (t in 1:T) {
            N[t]   ~ dpois(NN)
        }
        NN      ~ dunif(10,100)
        
        for (i in 1:cp) {
            b[i]   ~ dnorm(0, 10) I(0,)
        }
        
        for (i in 1:cp) {
            a[i]  <- b[i] / sum(b)
        }
}
"""

myvars=['NN', 'N', 'a']

%time model = pyjags.Model(code, data=dict(n=sample, cp=cp, T=T), chains=4, threads=4)
%time samples = model.sample(20000, vars=myvars)
samples.keys()

4 5
adapting: iterations 4000 of 4000, elapsed 0:00:00, remaining 0:00:00
CPU times: user 496 ms, sys: 28.4 ms, total: 525 ms
Wall time: 468 ms
sampling: iterations 1927 of 80000, elapsed 0:00:01, remaining 0:00:23
sampling: iterations 14067 of 80000, elapsed 0:00:01, remaining 0:00:05
sampling: iterations 21046 of 80000, elapsed 0:00:02, remaining 0:00:06
sampling: iterations 31119 of 80000, elapsed 0:00:03, remaining 0:00:04
sampling: iterations 39304 of 80000, elapsed 0:00:03, remaining 0:00:03
sampling: iterations 45517 of 80000, elapsed 0:00:04, remaining 0:00:03
sampling: iterations 55476 of 80000, elapsed 0:00:04, remaining 0:00:02
sampling: iterations 63750 of 80000, elapsed 0:00:05, remaining 0:00:01
sampling: iterations 68929 of 80000, elapsed 0:00:05, remaining 0:00:01
sampling: iterations 74606 of 80000, elapsed 0:00:06, remaining 0:00:00
sampling: iterations 80000 of 80000, elapsed 0:00:06, remaining 0:00:00
CPU times: user 8.69 s, sys: 665 ms, total: 9.36 s
Wall time: 6.2

dict_keys(['NN', 'N', 'a'])

In [3]:
burnin = 5000
        
for varname in myvars:
    summary(samples, varname, burnin=burnin, thin=4)

NN      [0] mean =      58.5322 (std =     4.7832), 95% credible interval [50.83 66.55]
N       [0] mean =      55.7821 (std =     5.7967), 95% credible interval [47.00 66.00]
N       [1] mean =      46.1269 (std =     5.5198), 95% credible interval [37.00 55.00]
N       [2] mean =      53.7649 (std =     5.7647), 95% credible interval [45.00 64.00]
N       [3] mean =      55.8573 (std =     5.8246), 95% credible interval [47.00 66.00]
N       [4] mean =      80.0723 (std =     6.5386), 95% credible interval [70.00 91.00]
a       [0] mean =       0.2418 (std =     0.0246), 95% credible interval [ 0.20  0.28]
a       [1] mean =       0.4167 (std =     0.0281), 95% credible interval [ 0.37  0.46]
a       [2] mean =       0.3110 (std =     0.0264), 95% credible interval [ 0.27  0.36]
a       [3] mean =       0.0304 (std =     0.0100), 95% credible interval [ 0.02  0.05]


## Now with some noise

In [145]:
sample = np.array([
        [10, 5, 22, 14, 130],
        [20, 14, 18, 22, 50],
        [20, 600, 18, 200, 25],
        [3, 2, 1, 7, 0],
        [500, 400, 1000, 300, 300],
        [60, 40, 20, 80, 800]
    ])

sample = np.array([
        [ 28, 330,  37,  32,  28,  23],
        [ 26,  35,  32,  25, 275,  23],
        [ 11,   9,   8,   9, 165,   4],
        [ 67,  76,  51, 605,  60,  63],
        [ 61, 627,  65,  53,  64,  61]
    ])

sample = gen_noised_poisson(12, gen_lambdas(30, 10), p_noise=0.05, i_noise=10)


cp, T = sample.shape

print(cp, T)

10 12


In [154]:
code = """
# bugs model for poisson distribs

model{
        for (i in 1:cp) {
            for (t in 1:T) {
                m[i, t]       ~ dpois(a[i] * N[t])
                noise[i, t]   ~ dbern(p_noise)
                outl_f[i, t]  ~ dt(0, 1.5, 4) I(0,)
                outl[i, t]    ~ dpois(N[t] * a[i] * (1 + outl_f[i, t]))
                mm[i, t, 1]  <- m[i, t]
                mm[i, t, 2]  <- outl[i, t] + 2 * a[i] * N[t]
                
                n[i, t]       ~ dnorm(mm[i, t, choice[i, t]], tau)
                choice[i, t] <- noise[i, t] + 1
            }
        }
        
        tau         <- 1 / sigma / sigma
        sigma        ~ dunif(0.0001, 0.001)
        
        for (t in 1:T) {
            N[t]   ~ dpois(NNN)
        }
        
        NNN      ~ dunif(NN/2, NN*2)
        p_noise  ~ dbeta(2, 40)
        
        for (i in 1:cp) {
            b[i]   ~ dunif(0, 1)
        }
        
        for (i in 1:cp) {
            a[i]  <- b[i] / sum(b)
        }
}
"""

myvars=['NNN', 'N', 'a', 'p_noise', 'noise', 'sigma']

%time model = pyjags.Model(code,  \
                           data=dict(n=sample, cp=cp, T=T, NN=sample.sum(axis=0).mean()),  \
                           chains=4, threads=4, adapt=1000)
%time samples = model.sample(4000, vars=myvars, monitor_type="trace")
samples.keys()

adapting: iterations 17 of 4000, elapsed 0:00:01, remaining 0:02:55
adapting: iterations 45 of 4000, elapsed 0:00:01, remaining 0:02:00
adapting: iterations 89 of 4000, elapsed 0:00:02, remaining 0:01:22
adapting: iterations 114 of 4000, elapsed 0:00:02, remaining 0:01:22
adapting: iterations 215 of 4000, elapsed 0:00:03, remaining 0:00:51
adapting: iterations 294 of 4000, elapsed 0:00:03, remaining 0:00:43
adapting: iterations 389 of 4000, elapsed 0:00:04, remaining 0:00:38
adapting: iterations 488 of 4000, elapsed 0:00:05, remaining 0:00:33
adapting: iterations 595 of 4000, elapsed 0:00:05, remaining 0:00:29
adapting: iterations 668 of 4000, elapsed 0:00:06, remaining 0:00:28
adapting: iterations 752 of 4000, elapsed 0:00:06, remaining 0:00:27
adapting: iterations 794 of 4000, elapsed 0:00:07, remaining 0:00:27
adapting: iterations 882 of 4000, elapsed 0:00:07, remaining 0:00:26
adapting: iterations 984 of 4000, elapsed 0:00:08, remaining 0:00:24
adapting: iterations 1071 of 4000, el

dict_keys(['sigma', 'p_noise', 'NNN', 'noise', 'N', 'a'])

In [155]:
for varname in myvars:
    if varname != 'noise':
        summary(samples, varname, burnin=0, thin=4)

NNN     [0] mean =    1164.2617 (std =   161.5225), 95% credible interval [1002.06 1413.31]
N       [0] mean =    1579.0000 (std =   522.5165), 95% credible interval [1215.00 2480.00]
N       [1] mean =    1863.2500 (std =   748.7955), 95% credible interval [1108.00 2915.00]
N       [2] mean =     890.6788 (std =   113.5042), 95% credible interval [751.00 1065.00]
N       [3] mean =     984.9578 (std =   118.0141), 95% credible interval [845.00 1144.00]
N       [4] mean =     932.0865 (std =    76.3480), 95% credible interval [834.00 1057.00]
N       [5] mean =     943.8703 (std =    78.6288), 95% credible interval [842.00 1067.00]
N       [6] mean =     900.4707 (std =   101.4395), 95% credible interval [781.00 1077.00]
N       [7] mean =     949.8742 (std =    76.1956), 95% credible interval [854.00 1077.00]
N       [8] mean =    1700.0642 (std =   280.5644), 95% credible interval [1480.00 2182.00]
N       [9] mean =    1283.5000 (std =   145.1456), 95% credible interval [1077.00 148

In [156]:
noise = samples["noise"]
(a, b, d, e) = noise.shape

noise.mean(axis=2).mean(axis=2)

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.25,  0.  ,  0.  ,  0.  ,  0.25,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.5 ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.25,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.25,  0.25,  0.25,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         1.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  

In [157]:
sample

array([[   0,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0],
       [  37,  352,   36,   26,   35,   33,   42,   37,   25,   38,   36,
          39],
       [ 245,  246,  238,  241,  226,  233,  257,  214,  242,  223,  219,
         217],
       [   9,    6,   12,    9,    7,   10,    9,    2,    9,   11,   88,
          12],
       [ 146,  120,  121,  138,  148,  139,  123,  151, 1474,  129,  151,
         161],
       [  61,   71,   85,   71,   78,   67,   84,   81,   81,   77,   77,
          66],
       [  74,   78,   73,   92,   74,   88,   96,   85,   93,   84,   73,
          92],
       [ 396,   32,   31,   27,   27,   30,   23,   37,   24,  407,   41,
          34],
       [ 126,  106,  100,  110,   84,   97,   92,  113,  121,  109,  123,
          94],
       [  36,   48,   45,   29,   41,   38,   30,   37,   48,   41,   34,
          37]])